In [24]:
import pandas as pd
import numpy as np
import ast

In [1]:
!which python

/opt/conda/envs/pytorch/bin/python


In [2]:
!pwd

/home/ubuntu/sky_workdir


In [5]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
# Download the dataset
# api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions', 'RAW_recipes.csv', unzip=True)

In [5]:
data = pd.read_csv('notebooks/RAW_recipes.csv')
data.head(2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6


In [6]:
# Select only 'name' and 'steps' columns 
data = data[['name', 'steps', 'description', 'ingredients']]
# shape 
data.shape

(231637, 4)

In [7]:
# Count number of rows that contain 'please ignore' or 'none' string 
print(data[data['steps'].str.contains('please ignore')].shape)
print(data[data['steps'].str.contains('none')].shape)

(1, 4)
(103, 4)


In [8]:
# Remove rows that contain 'please ignore' or 'none' string
data = data[~data['steps'].str.contains('please ignore')]
data = data[~data['steps'].str.contains('none')]
data.shape

(231533, 4)

In [9]:
def clean_steps(data):
    # Convert the string representation of a list to an actual list
    data_list = ast.literal_eval(data)
    # Join the list elements into a single string
    return ', '.join(data_list)

data['steps'] = data['steps'].apply(clean_steps)
data['ingredients'] = data['ingredients'].apply(clean_steps)
data.head(2)

,name,steps,description,ingredients
0,arriba baked winter squash mexican style,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,..."
1,a bit different breakfast pizza,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil..."


In [10]:
# assemble lines: concatenate 'steps' with 'name'
lines = data.apply(lambda row: str(row['name']) + 
                   '; ' + 
                   str(row['steps']).replace("\n", ' ') + 
                   '; ' + 
                   str(row['description']) + 
                   '; ' + 
                   str(row['ingredients']), axis=1).tolist()
sorted(lines, key=len)[:10]


['cherry coke; blend and chill then serve; a friend; coke, kool-aid',
 'chicken test; cook it, eat it; i like chicken; banana, apple, milk',
 'delicious water; pour the water, delicious; water; water, lemon wedge',
 'twin peaks martini; shake and strain; nan; vodka, whiskey, triple sec',
 'deluxe scrambled eggs; cook in a pan; good eggs; egg, bacon, tomatoes, spinach',
 'tahini substitution; mix well; good to know.; tahini, peanut butter, sesame oil',
 'cape cod; shake with ice and serve on the rocks; nan; vodka, lime, cranberry juice',
 "polar blast; mix, enjoy; great on a winter's night.; brandy, rum, vanilla ice cream",
 'homemade sour cream with yogurt; mix and use; easy peasy!; plain yogurt, lemon juice',
 'maple milk; combine in a glass; very easy and tasty drink to make.; milk, maple syrup']

In [13]:
# Clean the data. Use WordPunctTokenizer or something.
# Convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def clean_line(line):
    return ' '.join(tokenizer.tokenize(line.lower()))

lines = [clean_line(line) for line in lines]
sorted(lines, key=len)[:10]

['cherry coke ; blend and chill then serve ; a friend ; coke , kool - aid',
 'chicken test ; cook it , eat it ; i like chicken ; banana , apple , milk',
 'delicious water ; pour the water , delicious ; water ; water , lemon wedge',
 'twin peaks martini ; shake and strain ; nan ; vodka , whiskey , triple sec',
 'deluxe scrambled eggs ; cook in a pan ; good eggs ; egg , bacon , tomatoes , spinach',
 'tahini substitution ; mix well ; good to know .; tahini , peanut butter , sesame oil',
 'cape cod ; shake with ice and serve on the rocks ; nan ; vodka , lime , cranberry juice',
 'homemade sour cream with yogurt ; mix and use ; easy peasy !; plain yogurt , lemon juice',
 'maple milk ; combine in a glass ; very easy and tasty drink to make .; milk , maple syrup',
 "polar blast ; mix , enjoy ; great on a winter ' s night .; brandy , rum , vanilla ice cream"]

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

Such approximation is known under the name of _n-th order markov assumption_.

In [14]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines: List[str], n: int) -> DefaultDict[Tuple[str, ...], Counter]:
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    Consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    for line in tqdm(lines):
        tokens = line.split()
        tokens = [UNK]*(n-1) + tokens + [EOS]
        for i in range(len(tokens)-n+1):
            prefix = tuple(tokens[i:i+n-1])
            counts[prefix][tokens[i+n-1]] += 1

    return counts

counts = count_ngrams(lines, 3)



  0%|          | 0/231533 [00:00<?, ?it/s]

100%|██████████| 231533/231533 [00:53<00:00, 4341.02it/s]


In [16]:
# Check the counts for the prefix  ('I', 'love'). Limit the output to 10 elements
counts[('cup', 'of')].most_common(10)


[('the', 5036),
 ('water', 1300),
 ('sugar', 514),
 ('cheese', 489),
 ('flour', 412),
 ('milk', 328),
 ('batter', 273),
 ('hot', 273),
 ('coffee', 221),
 ('tea', 186)]

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [26]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        for prefix, next_tokens in counts.items():
            total_count = float(sum(next_tokens.values()))
            for next_token, count in next_tokens.items():
                self.probs[prefix][next_token] = count / total_count
        
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's train it on the whole dataset.

In [27]:
lm = NGramLanguageModel(lines, n=3)

  0%|          | 466/231533 [00:00<00:49, 4653.36it/s]

100%|██████████| 231533/231533 [00:52<00:00, 4372.27it/s]


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [28]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    probs = lm.get_possible_next_tokens(prefix)
    if temperature == 0:
        return max(probs, key=probs.get)
    else:
        probs = {k: v**(1/temperature) for k, v in probs.items()}
        s = sum(probs.values())
        probs = {k: v/s for k, v in probs.items()}
        return np.random.choice(list(probs.keys()), p=list(probs.values()))

Let's have fun with the model.

In [35]:
prefix = 'cover with' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=1)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

cover with melted chocolate over low heat for about 15 - 20 minutes , about 5 minutes , turn and cook for a late night snack . makes a lot , add them to a boil , then turn to coat and heat to low , keep in a second burner just now ( early summer , because the under side of the fruit , place steamed broccoli florets , orange juice , water , stir in lemon juice , dried parsley , salt , brush with more dressing , catalina dressing _EOS_


These samples are not fluent: it can be clearly seen that the model does not use long context, and relies only on a couple of tokens. The inability to use long contexts is the main shortcoming of n-gram models.

We see that greedy texts (pick up the most probable token at each step) are:

- shorter - the _eos_ token has high probability;
- very similar - many texts end up generating the same phrase.


### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [37]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    logprobs = []
    for line in tqdm(lines):
        tokens = line.split()
        logprob = 0.
        prefix = ''
        for token in tokens:
            logprob += np.log(lm.get_next_token_prob(prefix, token))
            prefix += ' ' + token
        logprob += np.log(lm.get_next_token_prob(prefix, EOS))
        logprobs.append(max(logprob, min_logprob))
    return np.exp(-np.mean(logprobs))

In [ ]:
lm1 = NGramLanguageModel(lines, n=1)
lm3 = NGramLanguageModel(lines, n=3)
lm10 = NGramLanguageModel(lines, n=10)

ppx1 = perplexity(lm1, lines)
ppx3 = perplexity(lm3, lines)
ppx10 = perplexity(lm10, lines)
ppx_missing = perplexity(lm3, ['the jabberwock with eyes of flame'])

print("Perplexities: \n1gram: %.3f\n3gram: %.3f\n10gram: %.3f" % (ppx1, ppx3, ppx10))

Now let’s measure the actual perplexity: we’ll split the data into train and test and score model on test data only.

In [ ]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel): 
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

### Kneser-Ney smoothing 

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

Task:
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
from collections import defaultdict
from typing import List, Tuple
import numpy as np

class KneserNeyLanguageModel:
    def __init__(self, n: int, delta: float = 0.75):
        self.n = n
        self.delta = delta
        self.counts = defaultdict(int)
        self.context_counts = defaultdict(int)
        self.contexts = set()
        self.vocab = set()
        self.lambdas = {}
        self.probs = {}

    def train(self, corpus: List[str]):
        for sentence in corpus:
            tokens = sentence.split()
            for i in range(len(tokens)):
                for j in range(1, self.n+1):
                    if i-j < 0:
                        continue
                    context = tuple(tokens[i-j:i])
                    self.context_counts[context] += 1
                    self.contexts.add(context)
                    self.counts[(context, tokens[i])] += 1
                    self.vocab.add(tokens[i])

        for context, word in self.counts:
            count = self.counts[(context, word)]
            context_count = self.context_counts[context]
            if context_count > 0:
                prob = max(count - self.delta, 0) / context_count
                self.probs[(context, word)] = prob

        for context in self.contexts:
            total = 0
            for word in self.vocab:
                if (context, word) in self.probs:
                    total += self.probs[(context, word)]
            norm = 1 / total if total > 0 else 1
            self.lambdas[context] = self.delta * len(self.vocab) / (context_count * norm)

    def get_prob(self, context: Tuple[str], word: str) -> float:
        if context in self.lambdas:
            return self.lambdas[context] * self.get_prob(tuple(list(context)[1:]), word) + \
                   (1 - self.lambdas[context]) * self.probs.get((context, word), 0)
        else:
            return self.probs.get((context, word), 0)

    def perplexity(self, corpus: List[str]) -> float:
        log_prob = 0
        total_words = 0
        for sentence in corpus:
            tokens = sentence.split()
            for i in range(len(tokens)):
                total_words += 1
                context = tuple(tokens[max(0, i-self.n+1):i])
                word = tokens[i]
                log_prob += np.log2(self.get_prob(context, word))
        return 2**(-log_prob / total_words)


In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, smoothing=<...>)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))